In [397]:
import csv
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sqlalchemy import *
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine

In [398]:
# Importing the csv

file1 = "clean_hawaii_measurements.csv"
file2 = "hawaii_stations.csv"

df1 = pd.read_csv(file1, dtype=object)
df2 = pd.read_csv(file2, dtype=object)

In [399]:
measurements = "clean_hawaii_measurements.csv"
stations = "hawaii_stations.csv"
df1.to_csv(measurements, index=False)
df2.to_csv(stations, index=False)

In [400]:
from sqlalchemy import Table, Column, Float, Integer, String, MetaData, ForeignKey, Numeric

In [401]:
# Database Setup 

engine = create_engine("sqlite:///hawaii.sqlite", echo = False) 
conn = engine.connect()

In [402]:
# Declare Measurement as a class
Base = declarative_base()

class Measurement(Base):
    __tablename__ = 'measurements'
  
    station = Column(String, primary_key=True)
    date = Column(String, primary_key=True)
    prcp = Column(Float)
    tobs = Column(Integer)
    
    def __repr__(self):
        return f"id={self.id}"

In [403]:
Measurement.__table__

Table('measurements', MetaData(bind=None), Column('station', String(), table=<measurements>, primary_key=True, nullable=False), Column('date', String(), table=<measurements>, primary_key=True, nullable=False), Column('prcp', Float(), table=<measurements>), Column('tobs', Integer(), table=<measurements>), schema=None)

In [404]:
# Declare Stations as a class
class Station(Base):
    __tablename__ = 'stations'
    
    extend_existing=True
    station = Column(String, primary_key=True)
    name = Column(String)
    latitude = Column(Numeric)
    longitude = Column(Numeric)
    elevation = Column(Numeric)
    
    def __repr__(self):
        return f"id={self.id}"
    
    
    


In [405]:
Station.__table__

Table('stations', MetaData(bind=None), Column('station', String(), table=<stations>, primary_key=True, nullable=False), Column('name', String(), table=<stations>), Column('latitude', Numeric(), table=<stations>), Column('longitude', Numeric(), table=<stations>), Column('elevation', Numeric(), table=<stations>), schema=None)

In [406]:
# create the tables
Base.metadata.create_all(engine)

In [407]:
# First, let's examine the tables in the db
tables = engine.execute("SELECT * FROM sqlite_master WHERE TYPE='table'")
for index, table in enumerate(tables):
    table_name = table[1]
    print(f"The #{index} table in the database is called '{table_name}'.")

The #0 table in the database is called 'sqlite_sequence'.
The #1 table in the database is called 'measurements'.
The #2 table in the database is called 'stations'.


In [408]:
conn = engine.connect()

In [409]:
stations = conn.execute("SELECT * FROM stations;").fetchall()

In [410]:
measuremens = conn.execute("SELECT * FROM measurements;").fetchall()

In [412]:
# To persist Student objects into, and load from, the database, 
# we use a Session object
# A session is akin to a conversation between Python and SQL
from sqlalchemy.orm import Session
# Declare a session
session = Session(bind=engine)

In [413]:
# Load the cleaned csv file into a pandas dataframe
### BEGIN SOLUTION
df1 = pd.read_csv(file1)
df2 = pd.read_csv(file2)
### END SOLUTION

In [414]:
# Use Orient='records' to create a list of data to write
# http://pandas-docs.github.io/pandas-docs-travis/io.html#orient-options
### BEGIN SOLUTION
data1 = df1.to_dict(orient='records')
data2 = df2.to_dict(orient='records')
data1[0]
### END SOLUTION

{'date': '2010-01-01', 'prcp': 0.08, 'station': 'USC00519397', 'tobs': 65}

In [415]:
# Use MetaData from SQLAlchemy to reflect the tables
### BEGIN SOLUTION
metadata1 = MetaData(bind=engine)
metadata1.reflect()
### END SOLUTION

In [416]:
metadata2 = MetaData(bind=engine)
metadata2.reflect()

In [417]:
# Save the reference to the `demographics` table as a variable called `table`
### BEGIN SOLUTION
table1 = sqlalchemy.Table('measurements', metadata1, autoload=True)
table2 = sqlalchemy.Table('stations', metadata2, autoload=True)
### END SOLUTION

In [418]:
conn.execute(table1.insert(), data1)

In [419]:
conn.execute(table2.insert(), data2)

In [420]:
session.commit()